In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import torch

start = time.time()
df = pd.read_csv('cleaned_puzzle_batches/cleaned_train_batch_1.csv')

output_df = df.filter(like='theme')
input_cols = [col_name for col_name in df.columns if not col_name.startswith('theme')]
input_df = df.filter(items=input_cols)

corr_df = output_df.apply(lambda x: input_df.corrwith(x))



end = time.time()
duration = end - start
print("exec time: %.4fs" % duration)

exec time: 4.9591s


In [7]:
torch.cuda.is_available()

/home/bastosgabriel/miniconda3/envs/chessenv/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554800319/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [2]:
class ChessData(torch.utils.data.Dataset):
    def __init__(self, path):
        self.df = pd.read_csv(path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = df.iloc[idx]

        inputs = row[:321].values
        outputs = row[321:].values

        return {'board': inputs, 'themes': outputs}


In [3]:
from torch.utils.data import DataLoader

chess_data_set = ChessData('cleaned_puzzle_batches/cleaned_train_batch_1.csv')

train_dataloader = DataLoader(chess_data_set, batch_size=64, shuffle=True)
print(train_dataloader)

# Display image and label.
batch = next(iter(train_dataloader))
print(f"batch['board']: {batch['board']}")
print(f"batch['themes']: {batch['themes']}")
print(f"batch['themes']: {batch['themes'].shape}")

batch['board']: tensor([[ 0.,  0.,  0.,  ..., -1.,  0.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  0., -1.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  ...,  0.,  0., -1.],
        [ 0.,  0.,  0.,  ..., -1.,  0., -1.]], dtype=torch.float64)
batch['themes']: tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)
batch['themes']: torch.Size([64, 27])


In [4]:
import torch.nn as nn

net = nn.Sequential(nn.Linear(321, 27))


In [5]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

loss = nn.BCEWithLogitsLoss()

In [6]:
for batch in train_dataloader:
    X = batch['board'].cuda()

RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW